<a href="https://colab.research.google.com/github/Dhayanshariff/MongoDB-Tasks/blob/main/Students%20Score%20Task/MongoDB_Students_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pymongo
!apt install mongodb > log
!service mongodb start
from pymongo import MongoClient
myClient = MongoClient()



 * Starting database mongodb
   ...done.


In [2]:
Mongo_Tasks_new = myClient['Mongo_Tasks_new']
Students = Mongo_Tasks_new['Students']

In [8]:
import json
import pprint

In [5]:
with open('Student.json') as file:
  file_data = json.load(file)

In [6]:
Students.insert_many(file_data)

In [7]:
# 1) Find the student name who scored maximum scores in all (exam, quiz and homework)?

def Find_Topper(type):
  stage1 = {"$unwind": "$scores"}
  stage2 = {"$match" : {"scores.type" : type}}
  stage3 = {"$sort" :{"scores.score": -1}}
  stage4 = {"$limit" : 1}
  stage5 = {"$project" : {"_id" : 0,"name":1,"scores.score":1,"scores.type" : 1}}

  for i in Students.aggregate([stage1, stage2,stage3,stage4,stage5]):
    print(i)


Find_Topper("quiz")
Find_Topper("exam")
Find_Topper("homework")

{'name': 'Cody Strouth', 'scores': {'score': 99.80348240553108, 'type': 'quiz'}}
{'name': 'Margart Vitello', 'scores': {'score': 99.33685767140612, 'type': 'exam'}}
{'name': 'Whitley Fears', 'scores': {'score': 99.77237745070993, 'type': 'homework'}}


In [18]:
# 3) Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories. 
# (If the student scores 40 or above in all types, then only they will be marked as pass. Else they will be marked as Fail)

for i in Students.find():
  if (i['scores'][0]['score']>=40 and i['scores'][1]['score']>=40 and i['scores'][2]['score']>=40):
    Students.update_one({'_id': i['_id']},{'$set': {'Result': 'Pass'}})
  else:
    Students.update_one({'_id': i['_id']},{'$set': {'Result': 'Fail'}})

for i in Students.find({},{'_id':1,'name':1,'Result':1}):
  print(i)

{'_id': 0, 'name': 'aimee Zank', 'Result': 'Fail'}
{'_id': 1, 'name': 'Aurelia Menendez', 'Result': 'Pass'}
{'_id': 2, 'name': 'Corliss Zuk', 'Result': 'Fail'}
{'_id': 3, 'name': 'Bao Ziglar', 'Result': 'Fail'}
{'_id': 4, 'name': 'Zachary Langlais', 'Result': 'Fail'}
{'_id': 5, 'name': 'Wilburn Spiess', 'Result': 'Fail'}
{'_id': 6, 'name': 'Jenette Flanders', 'Result': 'Fail'}
{'_id': 7, 'name': 'Salena Olmos', 'Result': 'Pass'}
{'_id': 8, 'name': 'Daphne Zheng', 'Result': 'Fail'}
{'_id': 9, 'name': 'Sanda Ryba', 'Result': 'Fail'}
{'_id': 10, 'name': 'Denisha Cast', 'Result': 'Pass'}
{'_id': 11, 'name': 'Marcus Blohm', 'Result': 'Pass'}
{'_id': 12, 'name': 'Quincy Danaher', 'Result': 'Pass'}
{'_id': 13, 'name': 'Jessika Dagenais', 'Result': 'Pass'}
{'_id': 14, 'name': 'Alix Sherrill', 'Result': 'Fail'}
{'_id': 15, 'name': 'Tambra Mercure', 'Result': 'Fail'}
{'_id': 16, 'name': 'Dodie Staller', 'Result': 'Fail'}
{'_id': 17, 'name': 'Fletcher Mcconnell', 'Result': 'Fail'}
{'_id': 18, 'na

In [19]:
# 4) Find the total and average of the exam, quiz and homework and store them in a separate collection.
#my understanding is that we need to find total and average for each student

# note: The output is stored in a new collection Total_and_Average_of_Students

Total_and_Average_of_Students = Mongo_Tasks_new['Total_Average_of_Students']

for i in Students.find():
    d={}
    e=i["scores"][0]["score"]
    q=i["scores"][1]["score"]
    h=i["scores"][2]["score"]
    total=e+q+h
    average=total/3
    d["_id"]=i["_id"]
    d["Name"]=i["name"]
    d["Total"]=total
    d["Average"]=average
    Total_and_Average_of_Students.insert_one(d)

for i in Total_and_Average_of_Students.find():
  print(i)

{'_id': 0, 'Name': 'aimee Zank', 'Total': 49.11994783171814, 'Average': 16.373315943906046}
{'_id': 1, 'Name': 'Aurelia Menendez', 'Total': 184.61969202100374, 'Average': 61.53989734033458}
{'_id': 2, 'Name': 'Corliss Zuk', 'Total': 139.61606947126907, 'Average': 46.53868982375636}
{'_id': 3, 'Name': 'Bao Ziglar', 'Total': 138.70712252233457, 'Average': 46.23570750744486}
{'_id': 4, 'Name': 'Zachary Langlais', 'Total': 203.3963625302728, 'Average': 67.79878751009093}
{'_id': 5, 'Name': 'Wilburn Spiess', 'Total': 134.0186975547794, 'Average': 44.67289918492647}
{'_id': 6, 'Name': 'Jenette Flanders', 'Total': 147.2203575476617, 'Average': 49.07345251588723}
{'_id': 7, 'Name': 'Salena Olmos', 'Total': 229.3959334774732, 'Average': 76.46531115915774}
{'_id': 8, 'Name': 'Daphne Zheng', 'Total': 112.71677331754347, 'Average': 37.57225777251449}
{'_id': 9, 'Name': 'Sanda Ryba', 'Total': 220.08328118626565, 'Average': 73.36109372875522}
{'_id': 10, 'Name': 'Denisha Cast', 'Total': 199.88435728

In [20]:
# 6) Create a new collection which consists of students who scored below the fail mark in all the categories.

# Note: the result is stored in new collection named 'Failed_in_all' and the new table entries are showed on screen for easy visibilty for user understanding

Failed_in_all = Mongo_Tasks_new['Failed_in_all']

for i in Students.find({'$and':[{'scores.0.score':{'$lt':40}},{'scores.1.score':{'$lt':40}},{'scores.2.score':{'$lt':40}}]},{'_id':1,'name':1}):
  Failed_in_all.insert_one(i)
  
for i in Failed_in_all.find():
  print(i)

{'_id': 0, 'name': 'aimee Zank'}


In [21]:
# 7) Create a new collection which consists of students who scored above pass mark in all the categories.

# Note: the result is stored in new collection named 'Passed_in_all' and the new table entries are showed on screen for easy visibilty for user understanding

Passed_in_all = Mongo_Tasks_new['Passed_in_all']

for i in Students.find({'$and':[{'scores.0.score':{'$gte':40}},{'scores.1.score':{'$gte':40}},{'scores.2.score':{'$gte':40}}]},{'_id':1,'name':1}):
  Passed_in_all.insert_one(i)
  
for i in Passed_in_all.find():
  print(i)

{'_id': 1, 'name': 'Aurelia Menendez'}
{'_id': 7, 'name': 'Salena Olmos'}
{'_id': 10, 'name': 'Denisha Cast'}
{'_id': 11, 'name': 'Marcus Blohm'}
{'_id': 12, 'name': 'Quincy Danaher'}
{'_id': 13, 'name': 'Jessika Dagenais'}
{'_id': 18, 'name': 'Verdell Sowinski'}
{'_id': 20, 'name': 'Tressa Schwing'}
{'_id': 23, 'name': 'Tamika Schildgen'}
{'_id': 32, 'name': 'Gwen Honig'}
{'_id': 36, 'name': 'Eugene Magdaleno'}
{'_id': 52, 'name': 'Tawana Oberg'}
{'_id': 54, 'name': 'Joel Rueter'}
{'_id': 55, 'name': 'Tresa Sinha'}
{'_id': 61, 'name': 'Grady Zemke'}
{'_id': 64, 'name': 'Whitley Fears'}
{'_id': 65, 'name': 'Gena Riccio'}
{'_id': 67, 'name': 'Merissa Mann'}
{'_id': 68, 'name': 'Jenise Mcguffie'}
{'_id': 75, 'name': 'Tonia Surace'}
{'_id': 79, 'name': 'Mariela Sherer'}
{'_id': 84, 'name': 'Timothy Harrod'}
{'_id': 85, 'name': 'Rae Kohout'}
{'_id': 90, 'name': 'Osvaldo Hirt'}
{'_id': 97, 'name': 'Maren Scheider'}
{'_id': 98, 'name': 'Carli Ector'}
{'_id': 99, 'name': 'Jaclyn Morado'}
{'_i

In [22]:
# 2)  Find students who scored below average in the exam and pass mark is 40%? 
# the question is not clear and my understanding is that requirement is to get the names of students who scored between 40 and Average of the all scores in type Exam
# I tried doing the same in single program but I could nto crack it. Hence i have split the code in to two, where i get the average in 1st step and remaining operation in next step.
import pprint
stage1={"$unwind":"$scores"}
stage2={'$match':{'scores.type':'exam'}}
stage3={"$group":{"_id":"$scores.type","avg_score":{"$avg":"$scores.score"}}} 


for i in Students.aggregate([stage1,stage2,stage3]): 
  pprint.pprint(i)

for i in Students.find({'scores.0.score':{'$gt':40,'$lte':{"$avg":"$scores.score"}}},{'_id':0,'name':1}):
  pprint.pprint(i)

{'_id': 'exam', 'avg_score': 48.67367075950175}


In [23]:
import pprint

for i in Students.find({'scores.0.score':{'$gt':40,'$lte':48.67367075950175}},{'_id':0,'name':1}):
  pprint.pprint(i)

{'name': 'Wilburn Spiess'}
{'name': 'Denisha Cast'}
{'name': 'Gisela Levin'}
{'name': 'Tressa Schwing'}
{'name': 'Rosana Vales'}
{'name': 'Tamika Schildgen'}
{'name': 'Gwyneth Garling'}
{'name': 'Kayce Kenyon'}
{'name': 'Terica Brugger'}
{'name': 'Chad Rahe'}
{'name': 'Jenise Mcguffie'}
{'name': 'Cassi Heal'}
{'name': 'Demarcus Audette'}
{'name': 'Flora Duell'}
{'name': 'Quincy Danaher'}
{'name': 'Alix Sherrill'}
{'name': 'Laureen Salomone'}
{'name': 'Gwyneth Garling'}
{'name': 'Aleida Elsass'}
{'name': 'Alica Pasley'}
{'name': 'Malisa Jeanes'}
{'name': 'Harriett Velarde'}
{'name': 'Mariela Sherer'}


In [52]:
# 5)      Create a new collection which consists of students who scored below average and above 40% in all the categories.

Stage1={"$unwind":"$scores"}
stage2={"$group":{"_id":"$scores.type","avg_score":{"$avg":"$scores.score"}}}
for i in Students.aggregate([stage1,stage2]): 
  pprint.pprint(i)

{'_id': 'homework', 'avg_score': 67.81869620661149}
{'_id': 'quiz', 'avg_score': 48.99672319430254}
{'_id': 'exam', 'avg_score': 48.67367075950175}


In [53]:
import pprint

for i in Students.find({'scores.0.score':{'$gt':40,'$lt':48.67367075950175}},{'_id':0,'name':1}):
  pprint.pprint(i)

for i in Students.find({'scores.1.score':{'$gt':40,'$lt':48.99672319430254}},{'_id':0,'name':1}):
  pprint.pprint(i)

for i in Students.find({'scores.1.score':{'$gt':40,'$lt':67.81869620661149}},{'_id':0,'name':1}):
  pprint.pprint(i)


{'name': 'Wilburn Spiess'}
{'name': 'Denisha Cast'}
{'name': 'Gisela Levin'}
{'name': 'Tressa Schwing'}
{'name': 'Rosana Vales'}
{'name': 'Tamika Schildgen'}
{'name': 'Gwyneth Garling'}
{'name': 'Kayce Kenyon'}
{'name': 'Terica Brugger'}
{'name': 'Chad Rahe'}
{'name': 'Jenise Mcguffie'}
{'name': 'Cassi Heal'}
{'name': 'Demarcus Audette'}
{'name': 'Flora Duell'}
{'name': 'Quincy Danaher'}
{'name': 'Alix Sherrill'}
{'name': 'Laureen Salomone'}
{'name': 'Gwyneth Garling'}
{'name': 'Aleida Elsass'}
{'name': 'Alica Pasley'}
{'name': 'Malisa Jeanes'}
{'name': 'Harriett Velarde'}
{'name': 'Mariela Sherer'}
{'name': 'Salena Olmos'}
{'name': 'Terica Brugger'}
{'name': 'Danika Loeffler'}
{'name': 'Joaquina Arbuckle'}
{'name': 'Rubie Winton'}
{'name': 'Brain Lachapelle'}
{'name': 'Osvaldo Hirt'}
{'name': 'Demarcus Audette'}
{'name': 'Mercedez Garduno'}
{'name': 'Karry Petrarca'}
{'name': 'Flora Duell'}
{'name': 'Salena Olmos'}
{'name': 'Tambra Mercure'}
{'name': 'Verdell Sowinski'}
{'name': 'Sadi